# 🧬 Training pipeline: Training ranking model </span>

In this notebook, you will train a ranking model using gradient boosted trees. 

In [1]:
%load_ext autoreload
%autoreload 2

import warnings

warnings.filterwarnings("ignore")

from recsys.config import settings
from recsys.data.preprocessing.splitting import train_test_split
from recsys.gcp.vertex_ai.serving.ranking import GCPRankingModel
from recsys.core.models.two_tower.ranking import (
    RankingModelFactory,
    RankingModelTrainer,
)x
from recsys.gcp.feature_store import client as fs_client
from recsys.gcp.bigquery import client as bq_client

In [ ]:
dict(settings)

## ☁️ Connect to Vertex AI Feature Online Store

In [ ]:
fs_client.initialize()
fos = fs_client.get_client()

## 💿 Create training dataset

In [4]:
trans_fv, articles_fv, customers_f, rankings_fv = fs_client.get_feature_views(fos)

In [ ]:
rankings_df = bq_client.fetch_feature_view_data(feature_view=rankings_fv)
trans_df = bq_client.fetch_feature_view_data(feature_view=trans_fv)
articles_df = bq_client.fetch_feature_view_data(
    feature_view=articles_fv,
    except_columns=["embeddings", "image_url", "article_description"],
)

In [6]:
rankings_df = rankings_df.join(
    trans_df.select(["customer_id", "month_sin", "month_cos"]),
    on="customer_id",
    how="left",
)

In [7]:
rankings_df = rankings_df.join(
    articles_df.select(["article_id", "colour_group_name"]),
    on="article_id",
    how="left",
)

In [8]:
rankings_df = rankings_df.drop("customer_id", "article_id")

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    df=rankings_df,
    test_size=settings.RANKING_DATASET_VALIDATION_SPLIT_SIZE,
)

In [ ]:
X_train.head(3)

In [ ]:
y_train.head(3)

# Training the ranking model

Let's train the ranking model:

In [12]:
model = RankingModelFactory.build()

In [ ]:
trainer = RankingModelTrainer(
    model=model, train_dataset=(X_train, y_train), eval_dataset=(X_val, y_val)
)

In [ ]:
trainer.fit()

In [ ]:
optimal_threshold, best_metrics = trainer.find_optimal_threshold(step=0.05)

## Evaluating the ranking model

Next, you'll evaluate how well the model performs on the validation data using metrics for classification such as precision, recall and f1-score:

In [ ]:
metrics = trainer.evaluate(log=True, threshold=optimal_threshold)

In [ ]:
metrics

It can be seen that the model has a low F1-score on the positive class (higher is better). The performance could potentially be improved by adding more features to the dataset, e.g. image embeddings.

Let's see which features your model considers important.

In [ ]:
trainer.get_feature_importance()

# 🗄️ Upload models to Vertex AI model registry

In [19]:
ranking_model_gcp = GCPRankingModel(model=model)

In [ ]:
ranking_model_vertex = ranking_model_gcp.upload_to_vertex_ai(
    model_name="ranking_model_v1",
    description="Ranking model for two-tower recommender system",
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.2-1:latest",
)

# Upload the custom ranking docker container to endpoint for online predictions

Make sure to build and deploy the ranking container into your Artifact Registry before running this part.

**recsys/gcp/vertex_ai/containers/ranking/build.sh**

In [29]:
from google.cloud import aiplatform

aiplatform.init(project=settings.GCP_PROJECT, location=settings.GCP_LOCATION)

endpoint = aiplatform.Endpoint(endpoint_name="recsys-rankings-model")

In [ ]:
deployment = endpoint.deploy(
    model=None,
    deployed_model_display_name="recsys-rankings-model",
    machine_type="n1-standard-2",
    min_replica_count=1,
    max_replica_count=1,
    traffic_percentage=100,
    image_uri="us-central1-docker.pkg.dev/recsys-dev-gonzo-1/recsys-dev-artifact-registry/recsys-ranking-predictor",
    predict_route="/predict",
    health_route="/health",    
    service_account="terraform-sa@recsys-dev-gonzo-1.iam.gserviceaccount.com",
)

